In [ ]:
#!pip install transformers
#!pip install --upgrade unsloth unsloth_zoo

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import time
import os

#os.environ["PYTORCH_CUDA_ALLOC_CONF = expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
# --- 1. Configuration ---
# You can adjust these parameters
MODEL_NAME = "gpt2-xl"
MAX_LENGTH = 1024 # Maximum sequence length for GPT-2 Medium
BATCH_SIZE = 4
LEARNING_RATE = 5e-5
EPOCHS = 1
NUM_WORKERS = 4 # For faster data loading
PATH= 'MAAI_gpt2_extra_large_torch-50'

# --- 2. Device Setup (Essential for PyTorch) ---
# Automatically detects and uses GPU (CUDA/ROCm) or falls back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch using device: {device}")


/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


PyTorch using device: cuda


In [3]:
!wget --no-check-certificate "https://jpdata1-my.sharepoint.com/:u:/g/personal/anand_jpdata_co/EfMPNio2KkZLpNLNYPN3VbwBgk8YWlAAxZBhZBYqpmKdkA?e=D14Who&download=1" -O MAAI_gpt2_medium_torch_50


--2025-10-09 13:46:11--  https://jpdata1-my.sharepoint.com/:u:/g/personal/anand_jpdata_co/EfMPNio2KkZLpNLNYPN3VbwBgk8YWlAAxZBhZBYqpmKdkA?e=D14Who&download=1
Resolving jpdata1-my.sharepoint.com (jpdata1-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8fa::10, ...
Connecting to jpdata1-my.sharepoint.com (jpdata1-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/anand_jpdata_co/Documents/gpt-weights/MAAI_gpt2-medium-torch-50?ga=1 [following]
--2025-10-09 13:46:11--  https://jpdata1-my.sharepoint.com/personal/anand_jpdata_co/Documents/gpt-weights/MAAI_gpt2-medium-torch-50?ga=1
Reusing existing connection to jpdata1-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 1444645498 (1.3G) [application/octet-stream]
Saving to: ‘MAAI_gpt2_medium_torch_50’

MAAI_gpt2_medium_to 100%[===================>]   1.34G   112MB/s    in 12s     

2025-10-09 13:46:23 (118 MB/s) - ‘MAAI_gpt2_medium_t

In [4]:

# --- 5. Initialization ---

# Load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Set the padding token, which is necessary for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the Model and move it to the device (GPU or CPU)
#model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Use the lines below for loading stored weights and comment the line above
model = torch.load(PATH, weights_only=False)



2025-10-09 13:46:29.655121: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-09 13:46:29.655733: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 13:46:29.658572: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-09 13:46:29.664680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760017589.675092   40549 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760017589.67

In [5]:
prompt = "तुमच्या"
max_new_tokens = 1024

# 1. Tokenize input and move to device
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)



In [6]:
# 2. Generate text
print(f"Generating continuation for prompt: '{prompt}'")
        # Use model.generate() for text generation
with torch.no_grad():
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=512, # Set max total length
    num_return_sequences=1,
    compile_config=None,
    do_sample=True, # Use sampling for more creative output
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id # Ensure generation stops correctly
    )

# 3. Decode and Print Output
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("-" * 20)
print(f"Generated Text:\n{generated_text}")
print("-" * 20)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating continuation for prompt: 'तुमच्या'


/opt/conda/envs/py_3.12/lib/python3.12/site-packages/transformers/integrations/sdpa_attention.py:83: UserWarning: Using AOTriton backend for Efficient Attention forward... (Triggered internally at /var/lib/jenkins/pytorch/aten/src/ATen/native/transformers/hip/attention.hip:1180.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


--------------------
Generated Text:
तुमच्या ताकदीवर अनेक संकटं पेलून उभा राहिलो आहे असे म्हणत मतदारांच्या दिलेल्या उत्तरावर विचारवंत सुरु असताना वैद्यकीय शिक्षण मंडळाचे आज अध्यक्ष आणि माजी मंत्री हर्षवर्धन पाटील यांची निवड करण्यात आली. पुढील शिक्षण मंडळावर मनोज जरांगे पाटील यांची प्रकृती खालावल्याने त्यांना संबोधित करण्याची विनंती करण्यात आली आहे. मात�
--------------------
